### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`** 완

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [1]:
import re

class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    
    #문제 1-1
    for sequence in sequences:
      preprocessing_sequence=re.sub(r'[^0-9a-zA-Z ]', "", sequence.lower()) # 대문자->소문자 변환, 특수문자 제거
      split_sequence=list(filter(None, preprocessing_sequence.split())) # white space를 기준으로 token화, None 데이터 삭제
      result.append(split_sequence)

    return result
  
  def fit(self, sequences):
    self.fit_checker = False

    #문제 1-2
    tokens = self.preprocessing(sequences) # Preprocessing 실행 및 결과 저장

    for token in tokens:
      for word in token:
        if word not in self.word_dict.keys(): # word_dict에 해당 word가 존재하는지 체크
          self.word_dict[word]=len(self.word_dict.keys()) # 존재하지 않을 경우 word_dict에 word를 저장하고 token index를 할당

    self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)

    if self.fit_checker:

      #문제 1-3
      for token in tokens:
        token_by_result=list()

        for word in token:
          if word in self.word_dict.keys(): # word_dict에 해당 word가 존재하는지 체크
            token_by_result.append(self.word_dict[word]) # 존재할 경우 해당 word를 token index로 반환
          else:
            token_by_result.append(self.word_dict['oov']) # 존재하지 않을 경우 해당 word는 oov로 취급하고 0 token index로 반환

        result.append(token_by_result)

      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [2]:
Tokenizer().fit_transform(['I go to school.', 'I LIKE pizza!'])

[[1, 2, 3, 4], [1, 5, 6]]

### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [3]:
import numpy as np

class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)

    #문제 2-1
    token_list=list(set(sum(tokenized, []))) # tokenized 결과를 하나의 list로 병합 및 uniq
    sentence_len=len(tokenized) # IDF 계산을 위한 입력된 문장의 수 계산
    self.df_matrix=list() # IDF 행렬 저장/반환을 위한 변수

    for token_num in token_list:
      df=0
      for tokens in tokenized: # 각 단어(token)별로 문장을 탐색
        if token_num in tokens: # 단어(token)가 해당 문장에 포함되어있을 경우 df 증가
          df += 1 
      
      self.df_matrix.append(np.log(sentence_len/(1+df))) # IDF 계산 및 결과 저장
    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)

      #문제 2-2
      token_list=list(set(sum(tokenized, []))) # tokenized 결과를 하나의 list로 병합 및 uniq
      self.tfidf_matrix = list() # TF-IDF 결과 저장/반환을 위한 변수

      for sentence_token in tokenized:
        sen_tfidf_list=[float(0) for _ in range(len(token_list))]
        for token in sentence_token:
          idf=self.df_matrix[token-1] # IDF 결과값 호출
          tf= sentence_token.count(token) # tf 계산
          sen_tfidf_list[token-1]=tf*idf # TF-IDF 계산 및 저장

        self.tfidf_matrix.append(sen_tfidf_list)
      return self.tfidf_matrix

    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [4]:
TfidfVectorizer(Tokenizer()).fit_transform(['I go to school.', 'I LIKE pizza!'])

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]]